In [1]:
import numpy as np
from itertools import permutations
np.set_printoptions(precision=3, suppress=True)

In [2]:
def get_results_old(stacks):
    num_players = len(stacks)
    totalChips = sum(stacks)
    totalFinish = np.zeros((num_players, 3))
  
    # get first place results
    for i in range(num_players):
        totalFinish[i][0] = stacks[i]/totalChips

    # get second place results
    for i in range(num_players): # i is first
        if stacks[i] > 0:
            tempTotal = totalChips - stacks[i]
            for j in range(num_players): # j is second
                if j != i:
                    totalFinish[j][1] += stacks[j]/tempTotal * totalFinish[i][0]
                  
    # get third place results
    for i in range(num_players): # i is first
        if stacks[i] > 0:
            for j in range(num_players): # j is second
                if j != i and stacks[j] > 0:
                    tempTotal = totalChips - stacks[i] - stacks[j]
                    for k in range(num_players): # k is third
                        if k != i and k != j:
                            totalFinish[k][2] += stacks[k]/tempTotal * totalFinish[i][0] * stacks[j]/(tempTotal+stacks[j])
                            # Probability of k getting third times probability of i 
                            # getting first times probability of j getting 2nd given 
                            # i got first.
    return totalFinish

In [4]:
stacks = [3, 2, 1, 1]
get_results_old(stacks)

array([[ 0.429,  0.314,  0.186],
       [ 0.286,  0.31 ,  0.257],
       [ 0.143,  0.188,  0.279],
       [ 0.143,  0.188,  0.279]])

http://www.cse.unsw.edu.au/~cs9417ml/Bayes/Pages/Joint_Probability.html

I think the below terminology might b slightly off, but it does give the right answer

For players 1,2,...,k-1,k in player_subset, prev is the (joint) probability
that player 1 finishes first, player 2 finishes second, ..., and player
k-1 finishes in place k-1. stacks[player]/remaining_chips is the 
conditional probability that player k finishes in place k given prev.
Thus the new joint probability is the product of prev and the conditional.

In [5]:
def g(stacks, num_places=None):
    '''
    marginal[i, j] is the probability that player i will finish in place j
    joint[(x1, x2, ..., xk)] is the probability that player x1 will finish in
    first place, x2 will finish in second place, ..., xk will finish in kth place
    '''
    if num_places is None: num_places = len(stacks) 
    num_players = len(stacks)
    total_chips = stacks.sum()
    joint = {}
    marginal = np.zeros((num_players, num_places))

    perms = (player_subset for perm_size in range(1, num_places+1)
             for player_subset in permutations(range(num_players), perm_size))
    for player_subset in perms:
        player, others = player_subset[-1], player_subset[:-1]
        remaining_chips = total_chips - np.take(stacks, others).sum()
        joint[player_subset] = joint.get(others, 1) * stacks[player]/remaining_chips
        marginal[player, len(player_subset)-1] += joint[player_subset]
    return joint, marginal

In [6]:
stacks = np.array([2, 1, 1])
joint, marginal = g(stacks)
print(marginal)
print(get_results_old(stacks))
%precision 3
joint

[[ 0.5    0.333  0.167]
 [ 0.25   0.333  0.417]
 [ 0.25   0.333  0.417]]
[[ 0.5    0.333  0.167]
 [ 0.25   0.333  0.417]
 [ 0.25   0.333  0.417]]


{(0,): 0.500,
 (0, 1): 0.250,
 (0, 1, 2): 0.250,
 (0, 2): 0.250,
 (0, 2, 1): 0.250,
 (1,): 0.250,
 (1, 0): 0.167,
 (1, 0, 2): 0.167,
 (1, 2): 0.083,
 (1, 2, 0): 0.083,
 (2,): 0.250,
 (2, 0): 0.167,
 (2, 0, 1): 0.167,
 (2, 1): 0.083,
 (2, 1, 0): 0.083}

In [7]:
stacks = np.array([2e8, 1e8, 1e4, 2, 1])
joint, marginal = g(stacks)
print(marginal)
print(get_results_old(stacks))

[[ 0.667  0.333  0.     0.     0.   ]
 [ 0.333  0.667  0.     0.     0.   ]
 [ 0.     0.     1.     0.     0.   ]
 [ 0.     0.     0.     0.666  0.333]
 [ 0.     0.     0.     0.333  0.667]]
[[ 0.667  0.333  0.   ]
 [ 0.333  0.667  0.   ]
 [ 0.     0.     1.   ]
 [ 0.     0.     0.   ]
 [ 0.     0.     0.   ]]


In [11]:
for subset, p in joint.items():
    if p > 1e-4:
        print(subset, '%.4f'%p)

(0, 1) 0.6666
(0,) 0.6666
(0, 1, 2, 4) 0.2221
(0, 1, 2) 0.6664
(1, 0, 2, 3) 0.2221
(1,) 0.3333
(0, 1, 2, 3) 0.4443
(1, 0, 2, 3, 4) 0.2221
(1, 0, 2, 4, 3) 0.1111
(1, 0) 0.3333
(1, 0, 2) 0.3332
(0, 1, 2, 3, 4) 0.4443
(0, 1, 2, 4, 3) 0.2221
(0, 1, 3, 2) 0.0001
(0, 1, 3) 0.0001
(0, 1, 3, 2, 4) 0.0001
(1, 0, 2, 4) 0.1111
